In [101]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [102]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, itertools
#print('We will use RAPIDS version',cudf.__version__)
import warnings
warnings.filterwarnings('ignore')

# train load

In [103]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-chunk-data-inparquet-format/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/cris_baseline/'

In [104]:
%%time
type_labels = {'clicks':0, 'carts':1, 'orders':2}

files = glob.glob(INPUT_DIR+'train_parquet/*')
train = pd.DataFrame()

#full train load
for file in tqdm(files):
  tmp_train = pd.read_parquet(file)
  train = pd.concat([train,tmp_train])

train['type'] = train['type'].map(type_labels)
train

  0%|          | 0/129 [00:00<?, ?it/s]

CPU times: user 5min 53s, sys: 2min 20s, total: 8min 14s
Wall time: 7min 21s


,session,aid,ts,type
0,0,1517085,1659304800025,0
1,0,1563459,1659304904511,0
2,0,1309446,1659367439426,0
3,0,16246,1659367719997,0
4,0,1781822,1659367871344,0
...,...,...,...,...
704199,12899776,1737908,1661723987073,0
704200,12899777,384045,1661723976974,0
704201,12899777,384045,1661723986800,0
704202,12899778,561560,1661723983611,0


#knn code参考 https://dse-souken.com/2021/03/25/ai-20/

In [105]:
#train大きすぎてpivot table作れないのでまずはサンプル1000
#ただ、ここのサンプル次第で後段セルのtrain_piv = train.pivot(index="aid",columns="session",values="type").fillna(0) でエラー出る
#ValueError: Index contains duplicate entries, cannot reshape　　 

train_sample = train.sample(1000).reset_index(drop=True)
train_sample

,session,aid,ts,type
0,6812968,1336761,1660237468506,0
1,2112084,1014796,1659473134292,0
2,10944118,1362208,1661092934676,0
3,7215110,993603,1660646863307,0
4,856760,109002,1660668369092,0
...,...,...,...,...
995,4294138,156109,1660246074925,0
996,3591960,696431,1660938842576,0
997,808411,407661,1661332725075,0
998,1392055,610202,1660167550589,0


In [106]:
#pivot作成

train_piv = train_sample.pivot(index="aid",columns="session",values="type").fillna(0)
train_piv

session,1312,2021,2690,5122,6694,8228,10704,14922,16095,17326,...,12579838,12592490,12613671,12630657,12768547,12801434,12827498,12847887,12859072,12870463
aid,,,,,,,,,,,,,,,,,,,,,
1376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1848867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1851339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
#sparce mtrixにする

from scipy.sparse import csr_matrix
df_sp = csr_matrix(train_piv.values)
df_sp

<988x999 sparse matrix of type '<class 'numpy.float64'>'
	with 99 stored elements in Compressed Sparse Row format>

#knn学習

In [108]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=20,algorithm= "brute", metric= "cosine")
model = knn.fit(df_sp)
model

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

#sample knn予測

In [110]:
sample_aid = 1376 
distance, indice = model.kneighbors(train_piv.iloc[train_piv.index== sample_aid].values.reshape(1,-1),n_neighbors=20)

In [111]:
distance

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]])

In [112]:
indice

array([[664, 673, 677, 675, 650, 663, 662, 661, 660, 659, 658, 657, 656,
        655, 654, 653, 652, 665, 676, 671]])

In [113]:
rec_aids = train_piv.loc[train_piv.index[indice.flatten()[1]],:] #参考codeだとエラーなのでlocに変更
rec_aids_sort = sorted(rec_aids.items(), reverse=True, key=lambda x:x[1])#reverse→降順、辞書のvalueをkeyにソート
rec_aids_20 = rec_aids_sort[:20]

In [114]:
rec_aids_20

[(924584, 1.0),
 (1312, 0.0),
 (2021, 0.0),
 (2690, 0.0),
 (5122, 0.0),
 (6694, 0.0),
 (8228, 0.0),
 (10704, 0.0),
 (14922, 0.0),
 (16095, 0.0),
 (17326, 0.0),
 (24868, 0.0),
 (25141, 0.0),
 (25523, 0.0),
 (27695, 0.0),
 (45803, 0.0),
 (57751, 0.0),
 (58454, 0.0),
 (58467, 0.0),
 (59845, 0.0)]